<a href="https://colab.research.google.com/github/1908Deepak/Disease-Prediction/blob/main/option_chain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installing Dependencies**

In [1]:
!pip install upstox-python-sdk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.6/119.6 kB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for upstox-python-sdk: filename=upstox_python_sdk-2.9.0-py3-none-any.whl size=405208 sha256=ba83d2e3672d5fe5b0fdd5acca189d937d0953f5b1a3d2b62004144e04dbfb2d
  Stored in directory: /root/.cache/pip/wheels/52/d7/c3/35a06946c3312bcb7bba346c8e7e4b72f9f6a0a61fef567502
  Created wheel for uuid: filename=uuid-1.30-py3-none-any.whl size=6478 sha256=8350bf33484d5d392af89740ba427ea49bd8d46856d607b1eccf947af14dabe1
  Stored in directory: /root/.cache/pip/wheels/ed/08/9e/f0a977dfe55051a07e21af89200125d65f1efa60cbac61ed88
Successfully built upstox-python-sdk uuid


**Importing dependencies**

In [22]:
import pandas as pd
import requests
import os

**Login credentials to access Upstox API**

In [2]:
api_key = 'ed2bc706-c04c-4900-8a5f-67d3907e6801'   #enter your api_key
secret_key = 'lalnc4yki5'   #  enter your secret_key
redirect_url = 'https://api.upstox.com/v2/login'
#Generate authorization URL required for the OAuth process with Upstox.
url = f"https://api.upstox.com/v2/login/authorization/dialog?response_type=code&client_id={api_key}&redirect_uri={redirect_url}"
print(url)

https://api.upstox.com/v2/login/authorization/dialog?response_type=code&client_id=ed2bc706-c04c-4900-8a5f-67d3907e6801&redirect_uri=https://api.upstox.com/v2/login


**Authorization code**

In [3]:
# login code from authorization URL
code = 'I641qi'

**Exchange the authorization code for an access token in the OAuth 2.0 authentication flow.**

In [4]:
# Exchange the authorization code for an access token in the OAuth 2.0 authentication flow.
token_url = 'https://api.upstox.com/v2/login/authorization/token'
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/x-www-form-urlencoded'
}
data = {
    'code': code,
    'client_id': api_key,
    'client_secret': secret_key,
    'redirect_uri': 'https://api.upstox.com/v2/login',
    'grant_type': 'authorization_code',
}
response = requests.post(token_url, headers=headers, data=data)

# Check the status code of the response
if response.status_code == 200:
    json_response = response.json()
    # variable to store access token
    access_token = json_response['access_token']
else:
    print(f"Error: {response.status_code}")
    print(response.text)  # Print the error message from the API
    # Handle the error appropriately, e.g., retry with a new authorization code or check your credentials

**To fetch user's Profile**

In [5]:
profile_url = 'https://api.upstox.com/v2/user/profile'
headers = {
    'Accept': 'application/json',
    'Authorization': f'Bearer {access_token}'
}
response = requests.get(profile_url, headers=headers)

profile_details_json = response.json()
profile_details = pd.DataFrame(profile_details_json)
print(profile_details)

              status                       data
email        success        ds0709349@gmail.com
exchanges    success  [BSE, MCX, CDS, NSE, NFO]
products     success            [OCO, D, CO, I]
broker       success                     UPSTOX
user_id      success                     5YBXKE
user_name    success               DEEPAK SINGH
order_types  success  [MARKET, LIMIT, SL, SL-M]
user_type    success                 individual
poa          success                      False
ddpi         success                      False
is_active    success                       True


 """
    **Get option chain data from Upstox API and organizes it by strike price with bid/ask based on side.**

    Parameters::
    - instrument_key (str): The key of the instrument (e.g., 'NSE_INDEX|Nifty 50' or 'NSE_INDEX|Bank Nifty').
    - expiry_date (str): Expiration date in 'YYYY-MM-DD' format.
    - side (str): Specify 'PE' for put options, 'CE' for call options, or 'ALL' for both.

    Returns:
    - pd.DataFrame: DataFrame with columns: instrument_name, strike_price, side, and bid/ask.
    

In [23]:
def get_option_chain_data(instrument_key, expiry_date, side):
    # Determine instrument name based on instrument_key
    instrument_name = "NIFTY" if "Nifty 50" in instrument_key else "BANKNIFTY"

    url = 'https://api.upstox.com/v2/option/chain'
    params = {
        'instrument_key': instrument_key,
        'expiry_date': expiry_date
    }
    headers = {
        'Accept': 'application/json',
        'Authorization': f'Bearer {access_token}'  # Access token assumed to be defined globally
    }

    # Fetch the option chain data
    response = requests.get(url, params=params, headers=headers)

    # Check if the response was successful
    if response.status_code != 200:
        raise Exception(f"Error fetching data: {response.status_code}, {response.text}")

    option_chain_data = response.json().get('data', [])

    # Prepare a list to collect each row of data
    data = []

    # Process the option chain data
    for entry in option_chain_data:
        strike_price = entry['strike_price']

        # Get put (PE) data if required
        if side in ["PE", "ALL"]:
            put_data = entry.get('put_options', {})
            bid_price = put_data.get('market_data', {}).get('bid_price')
            if bid_price is not None:
                data.append({
                    'instrument_name': instrument_name,
                    'strike_price': strike_price,
                    'side': 'PE',
                    'bid/ask': bid_price
                })

        # Get call (CE) data if required
        if side in ["CE", "ALL"]:
            call_data = entry.get('call_options', {})
            ask_price = call_data.get('market_data', {}).get('ask_price')
            if ask_price is not None:
                data.append({
                    'instrument_name': instrument_name,
                    'strike_price': strike_price,
                    'side': 'CE',
                    'bid/ask': ask_price
                })

    # Convert to DataFrame
    df = pd.DataFrame(data, columns=['instrument_name', 'strike_price', 'side', 'bid/ask'])
    return df

print("Sample Output:")
print(get_option_chain_data('NSE_INDEX|Nifty 50', '2025-06-26', 'ALL'))


Sample Output:
   instrument_name  strike_price side  bid/ask
0            NIFTY       18000.0   PE    20.50
1            NIFTY       18000.0   CE  8200.00
2            NIFTY       19000.0   PE    22.80
3            NIFTY       19000.0   CE     0.00
4            NIFTY       20000.0   PE    31.95
5            NIFTY       20000.0   CE     0.00
6            NIFTY       21000.0   PE    59.95
7            NIFTY       21000.0   CE     0.00
8            NIFTY       22000.0   PE   206.05
9            NIFTY       22000.0   CE     0.00
10           NIFTY       23000.0   PE   365.00
11           NIFTY       23000.0   CE     0.00
12           NIFTY       24000.0   PE   585.00
13           NIFTY       24000.0   CE  1815.25
14           NIFTY       25000.0   PE   900.00
15           NIFTY       25000.0   CE  1199.95
16           NIFTY       26000.0   PE  1384.30
17           NIFTY       26000.0   CE   749.95
18           NIFTY       27000.0   PE  1911.30
19           NIFTY       27000.0   CE   403.9

**Calculates margin required and premium earned for each option in the DataFrame**

    Parameters:
    - df (pd.DataFrame): DataFrame containing option data with columns: instrument_name, strike_price, side, bid/ask.
    - num_contracts (int): Number of contracts traded (default is 1).

    Returns:
    - pd.DataFrame: Updated DataFrame with margin_required and premium_earned columns.


In [24]:
def calculate_margin_and_premium(df, num_contracts=1):
    # Calculate margin required (10% of bid/ask price for simplicity)
    df['margin_required'] = df['bid/ask'] * 0.10  # Example: 10% of the bid/ask price

    # Calculate premium earned (bid/ask price multiplied by number of contracts)
    df['premium_earned'] = df['bid/ask'] * num_contracts

    return df

# Example usage after fetching option chain data
instrument_key = 'NSE_INDEX|Nifty 50'
expiry_date = '2025-06-26'
side = 'ALL'  # 'PE' for put options, 'CE' for call options, or 'ALL' for both

# Fetching option chain data
df = get_option_chain_data(instrument_key, expiry_date, side)

# Calculating margin and premium
updated_df = calculate_margin_and_premium(df, num_contracts=1)
print("Sample Output:")

# Display the updated DataFrame
print(updated_df.to_string(index=False))

Sample Output:
instrument_name  strike_price side  bid/ask  margin_required  premium_earned
          NIFTY       18000.0   PE    20.50            2.050           20.50
          NIFTY       18000.0   CE  8200.00          820.000         8200.00
          NIFTY       19000.0   PE    22.80            2.280           22.80
          NIFTY       19000.0   CE     0.00            0.000            0.00
          NIFTY       20000.0   PE    31.95            3.195           31.95
          NIFTY       20000.0   CE     0.00            0.000            0.00
          NIFTY       21000.0   PE    59.95            5.995           59.95
          NIFTY       21000.0   CE     0.00            0.000            0.00
          NIFTY       22000.0   PE   206.05           20.605          206.05
          NIFTY       22000.0   CE     0.00            0.000            0.00
          NIFTY       23000.0   PE   365.00           36.500          365.00
          NIFTY       23000.0   CE     0.00            0.000 

**Clear previous data and save the new DataFrame to a CSV file.**
    
    Parameters:
    - df (pd.DataFrame): The DataFrame containing data to be saved.
    - file_path (str): The path of the CSV file.
    

In [21]:
def save_to_csv(df, file_path):
    # Check if the file exists and remove it to clear old data
    if os.path.exists('/content/premium.csv'):
        os.remove('/content/premium.csv')
        print(f"Cleared previous contents of {'/content/premium.csv'}")

    # Save the new data to the CSV file
    df.to_csv('/content/premium.csv', index=False)
    print(f"Data saved to {'/content/premium.csv'}")

# Example usage
premium_margin_file = '/content/premium.csv'

# Assume `df` is the DataFrame from your calculate_margin_and_premium function
# df = calculate_margin_and_premium(option_chain_df)

# Save the DataFrame to the CSV file after clearing previous data
save_to_csv(df, premium_margin_file)


Cleared previous contents of /content/premium.csv
Data saved to /content/premium.csv


**Sample Input 1:**

*   NSE_INDEX|Nifty Bank
*   2024-11-06



**Sample Input 2:**


*   NSE_INDEX|Nifty 50
*   2025-06-26

